# Black Scholes demo with NumPy & VTune

In [ ]:
# Boilerplate for the example
import numpy as np

try:
    import numpy.random_intel as rnd
except:
    import numpy.random as rnd

# make xrange available in python 3
try:
    xrange
except NameError:
    xrange = range
    
#Black Scholes dummy parameter

SEED = 7777777
S0L = 10.0
S0H = 50.0
XL = 10.0
XH = 50.0
TL = 1.0
TH = 2.0
RISK_FREE = 0.1
VOLATILITY = 0.2
TEST_ARRAY_LENGTH = 1024

###############################################

def gen_data(nopt):
    return (
        rnd.uniform(S0L, S0H, nopt),
        rnd.uniform(XL, XH, nopt),
        rnd.uniform(TL, TH, nopt),
        )

In [ ]:
nopt=100000
price, strike, t = gen_data(nopt)

## The Naive Black Scholes algorithm (looped)

Using Python lists and explicit loop.

In [ ]:
from math import log, sqrt, exp, erf
invsqrt = lambda x: 1.0/sqrt(x)

def black_scholes(nopt, price, strike, t, rate, vol):
    call = [0.0] * nopt
    put = [-1.0] * nopt
    mr = -rate
    sig_sig_two = vol * vol * 2
    
    for i in range(nopt):
        P = float( price[i] )
        S = strike[i]
        T = t[i]
        
        a = log(P / S)
        b = T * mr
        
        z = T * sig_sig_two
        c = 0.25 * z
        y = invsqrt(z)
        
        w1 = (a - b + c) * y
        w2 = (a - b - c) * y
        
        d1 = 0.5 + 0.5 * erf(w1)
        d2 = 0.5 + 0.5 * erf(w2)
        
        Se = exp(b) * S
        
        call[i] = P * d1 - Se * d2
        put[i] = call[i] - P + Se
        
    return (call, put)

In [ ]:
%timeit black_scholes(nopt, price, strike, t, RISK_FREE, VOLATILITY)

## The NumPy modified naive Black Scholes algorithm (looped)

Replacing Python lists with nd-arrays and using NumPy functions.

In [ ]:
#from numpy import log, sqrt, exp, erf, invsqrt

def black_scholes(nopt, price, strike, t, rate, vol):
    call = np.zeros(nopt, dtype=np.float64)
    put  = -np.ones(nopt, dtype=np.float64)
    mr = -rate
    sig_sig_two = vol * vol * 2
    
    for i in range(nopt):
        P = price[i]
        S = strike[i]
        T = t[i]
        
        a = np.log(P / S)
        b = T * mr
        
        z = T * sig_sig_two
        c = 0.25 * z
        y = np.invsqrt(z)
        
        w1 = (a - b + c) * y
        w2 = (a - b - c) * y
        
        d1 = 0.5 + 0.5 * np.erf(w1)
        d2 = 0.5 + 0.5 * np.erf(w2)
        
        Se = np.exp(b) * S
        
        call[i] = P * d1 - Se * d2
        put[i] = call[i] - P + Se
        
    return (call, put)

In [ ]:
%timeit black_scholes(nopt, price, strike, t, RISK_FREE, VOLATILITY)

## Performance as expected?

We should to work on NumPy Arrays to achieve vectorization.

Remove the loop and let it work on whole arrays!

In [ ]:
def black_scholes(nopt, price, strike, t, rate, vol):
    # No need to create/init put/call arrays
    mr = -rate
    sig_sig_two = vol * vol * 2

#TODO add non-looped code
    
    return (call, put)

In [ ]:
%timeit black_scholes(nopt, price, strike, t, RISK_FREE, VOLATILITY)

## Now let's do a detailed performance analysis

Use provided alias from amplxe.ipy to run VTune/amplifier.

In [ ]:
nopt=10000000
price, strike, t = gen_data(nopt)

call, put = black_scholes(nopt, price, strike, t, RISK_FREE, VOLATILITY)

In [ ]:
%pycat amplxe.ipy
%run amplxe.ipy

In [ ]:
# save python code to file
#TODO select the right cells!
%save -f runme.py 1 7 9
# collect data
%amplxe runme.py